
# Fake News Generator & Detector using GPT-2 and BERT with Gradio

---



This notebook allows you to generate fake news using GPT-2 and detect whether it's fake or real using a public BERT-based classifier.



**Project Description:**
A dual-function AI web app that:
- 🛠️ Generates fake news using GPT-2
- 🔍 Detects whether a news article is fake or real using BERT

This project combines the creative generation power of **GPT-2** with the classification accuracy of **BERT**, wrapped in an interactive **Gradio** interface.
**Features:**
* **Fake News Generation** based on user-provided prompts
* **Real/Fake Detection** of generated or custom news articles
* Built using **Transformers**, **PyTorch**, and **Gradio**   
* Easy-to-use, browser-based interface  





**Step 1: Install Required Libraries**

In [1]:

!pip install transformers torch gradio --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.4 MB/s eta 0:00:00


**Step 2: Import Libraries**

In [2]:

from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import gradio as gr


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

**Step 3: Load GPT-2 Model**

In [4]:

gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

**Step 4: Load BERT Model**

In [5]:
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Step 5: Generate Fake News**

In [6]:

def generate_fake_news(prompt):
    inputs = gpt2_tokenizer.encode(prompt, return_tensors="pt").to(device)
    outputs = gpt2_model.generate(
        inputs,
        max_length=200,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        early_stopping=True
    )
    generated_text = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

**Step 6: Detect Fake News**

In [7]:

def detect_news(text):
    inputs = bert_tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    confidence = torch.softmax(logits, dim=1)[0][predicted_class].item()
    label = "🟥 Fake News" if predicted_class == 0 else "🟩 Real News"
    return f"{label} (Confidence: {confidence:.2f})"


**Step 7: Gradio Interface**

In [9]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## 📰 Fake News Generator & Detector (GPT-2 + BERT)")

    with gr.Row():
        # Column for Fake News Generation
        with gr.Column():
            input_text = gr.Textbox(
                label="Enter a News Headline or Prompt",
                placeholder="e.g. A mysterious object was spotted in the sky...",
                lines=2
            )

            generate_btn = gr.Button("Generate Fake News")

            output_text = gr.Textbox(label="Generated News Article", lines=5)

        # Column for Detection
        with gr.Column():


            detect_input = gr.Textbox(
                label="Enter a News Article or Statement",
                placeholder="Paste a paragraph to detect if it's fake or real...",
                lines=5
            )
            detect_btn = gr.Button("Detect if Generated News is Fake or Real")
            detect_output = gr.Textbox(label="Detection Result")

    # Button actions
    generate_btn.click(fn=generate_fake_news, inputs=input_text, outputs=output_text)
    detect_btn.click(fn=detect_news, inputs=output_text, outputs=detect_output)

demo.launch()




It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2a87d6dcf37db0f546.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
